# NHL Prediction Project Data Wrangling

We have some great statistical data for the NHL games from the 2014, 2015, 2016, 2017, and 2018 seasons from Natural Stattrick as well from the Kaggle NHL dataset. The problem is that these stats are on a game by game basis, and that isn't very useful for prediction.

The purpose of this notebook is to wrangle these statistics into meaningful information that we can use to predict from.

We are going to 

## Game Data from NHL Database

We will start with getting the game data the Kaggle NHL Database. The only real reason to do this is to get game id's that we can then use to merge the Natural Stat Trick data with the gambling data. We could also just assign our own game id, which, may be easier.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
from datetime import timedelta
from tqdm import tqdm

%matplotlib inline
pd.options.display.width = 0
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
path = "../data/original/kaggle/"

teams = pd.read_csv(path + "team_info.csv")
games_all = pd.read_csv(path + "game.csv")
team_stats = pd.read_csv(path + "game_teams_stats.csv")

In [3]:
#display(teams.head())
def get_games(season):
    # convert season into a season number in the same format as the datafile
    season = season * 10000 + (season + 1)
    
    # filter for regular season games of that season
    games = games_all[(games_all['season'] == season) & (games_all['type'] == 'R')].copy()

    # create a small function to merge, drop, and rename
    def merge_games(df, team):
        x = (pd
             .merge(left = df,
                    right = teams[['team_id','teamName']],
                    left_on = team + '_id',
                    right_on = 'team_id')
             .drop(columns = ['team_id'])
             .rename(columns = {'teamName':team}))
        return x
          
    games = merge_games(games, 'away_team')
    games = merge_games(games, 'home_team')
    
    #convert datatypes
    games['date_time'] = pd.to_datetime(games['date_time'], format = '%Y-%m-%d')
    games['outcome'] = games['outcome'].astype('category')
    
    #filter for useful columns
    cols = ['game_id', 'season', 'date_time', 'away_team', 'home_team',
            'away_goals', 'home_goals', 'outcome', 'home_team_id', 'away_team_id']
    games_reduced = games[cols].sort_values(by='game_id')
    
    return games_reduced

In [4]:
g2014 = get_games(2014)
g2015 = get_games(2015)
g2016 = get_games(2016)
g2017 = get_games(2017)
g2018 = get_games(2018)

Run a quick quality check on one of the dataframes to look for duplications.

In [5]:
dup_cols = ['home_team', 'home_goals', 'away_team', 'away_goals', 'date_time']
g2016[g2016.duplicated(subset=dup_cols, keep = False)].sort_values(by='home_team')

,game_id,season,date_time,away_team,home_team,away_goals,home_goals,outcome,home_team_id,away_team_id


## Natural Stat Trick Data

Now, it is time to deal with the Natural Stat Trick Data

Create a number of procedures to make the cleaning more compartmentatlized and repeatable.

Skip to the next cell to actually get going.

In [6]:
def natstat_to_numeric(df_in):
    # '-' were causing some columns to be treated as strings so let's replace and convert
    df = df_in.drop(columns = ["Unnamed: 2", "Attendance"]).replace('-', np.nan)
    df.loc[:,'TOI':] = df.loc[:,'TOI':].apply(pd.to_numeric)
    
    
    #########################################################################################
    # we have to do something so let's fill with mean as we are calculating means
    #########################################################################################
    df = df.fillna(df.mean())
    
    # calculate blocked shots
    df['blocks'] = df['CA'] - df['FA']

    return df


def natstat_split_game_info(df_in):
    '''Split "Game" column into date, home team, away team, home goals, away goals'''
    df = df_in.copy()
    
    # get date and add it to the dataframe
    temp = df['Game'].str.split(' - ', n=1, expand = True)
    
    # temp[0] contains the date
    df.insert(0, 'date', pd.to_datetime(temp[0], format = '%Y-%m-%d'))
    
    # temp[1] contains the game information, but it is messy
    temp = temp[1].str.split(',', n=1, expand = True)
    
    # now temp[0] contains the away team info and temp[1] contains the home team info
    df.insert(1, 'home', temp[1].str.replace('\d+','').str.strip())
    df.insert(2, 'home_goals', temp[1].str.extract(r'(\d+)').astype('int64'))
    df.insert(3, 'away', temp[0].str.replace('\d+','').str.strip())
    df.insert(4, 'away_goals', temp[0].str.extract(r'(\d+)').astype('int64'))
    
    # in case we want to easily filter for home and away games we set H or A for each row
    hoa = ['H' if home in name else 'A' for home, name in zip(df['home'], df['Team'])]
    df.insert(5, 'hoa', hoa)
    
    # in case we want to easily filter for home and away games we set H or A for each row
    winner = []
    for h, a, t in zip(df['home_goals'], df['away_goals'], df['TOI']):
        result = ('H_reg' if h > a else 'A_reg' if t <= 60.0 else 
                  'H_OT' if h > a else 'A_OT' if t < 65.0 else
                  'H_SO' if h > a else 'A_SO')
        winner.append(result)
    df.insert(6, 'result', winner)
    df.insert(7, 'result_bool', df.result.apply(lambda x: 1 if x[0] == 'H' else 0))
    
    return df


def natstat_clean(df_in):
    ''' apply the cleaning functions to all data frames'''
    df = natstat_split_game_info(df_in)
    df = natstat_to_numeric(df)
    
    return df


def natstat_add_ave_stats(df_in, cols = None):
    ''' we want to create running averages for all of the numeric values of interest'''
    df = df_in.copy()
    df = df.sort_values(by='date')
    
    # get cols to create averages of
    if cols is None:
        cols = df.iloc[:, (df.columns.get_loc('TOI') + 1):].columns

    for c in cols:
        # create a column header name
        new_col = c + '_avg'
        
        # create the new column and fill with 0.0
        df[new_col] = 0.0
        
        # filter for each team and create the running average for each team individually
        for team in df.Team.sort_values().unique():
            f = df['Team'] == team
            df.loc[f, new_col] = df.loc[f, c].expanding(min_periods=2).mean().shift(1)
            
    return df


def natstat_home_away(df_in, hoa, col_suffix):
    '''
        Filter the original dataframe for home team or away team information that can
        be used to flatten each game
        
        Return two dataframes:
            header = data, home, away, home_goals, away_goals, hoa, Game, Team
            stats = all of the raw data
        
    '''
    df = df_in[df_in['hoa'] == hoa].copy()
    
    header = df.loc[:, 'date':'TOI'].reset_index(drop=True)
    stats = df.iloc[:, (df.columns.get_loc('TOI') + 1):].reset_index(drop=True)

    header.columns = [col + '_' + col_suffix for col in header.columns]
    stats.columns = [col + '_' + col_suffix for col in stats.columns]
    
    return header, stats


def natstat_flatten(df_in):
    ''' combine all stats into a single row for each game  '''
    h_head, h_stats = natstat_home_away(df_in, 'H', 'home')
    a_head, a_stats = natstat_home_away(df_in, 'A', 'away')
    
    h_head.columns = np.char.replace(h_head.columns.values.astype(str), '_home', '')
    
    return pd.concat([h_head, h_stats, a_stats], axis = 1)


def add_game_ids(nat_stat, nhl_df):
    ''' Add the NHL Game ID (from the Kaggle files) to the Nat Stat Data '''
    result = nat_stat.copy()
    game_ids = []
    
    for index, row in nat_stat.iterrows():
        try:
            f = ((nhl_df.home_team == row.home) &
                (nhl_df.away_team == row.away) &
                (nhl_df.home_goals == row.home_goals) &
                (nhl_df.away_goals == row.away_goals) &
                (
                    (nhl_df.date_time >= (row.date - timedelta(days=1))) &
                    (nhl_df.date_time <= (row.date + timedelta(days=1)))
                ))
            game_id = nhl_df[f].game_id.values[0]
            game_ids.append(game_id)
        except Exception as e:
            x = nhl_df[ (nhl_df.home_team == row.home) &
                        (nhl_df.away_team == row.away) &
                        (nhl_df.home_goals == row.home_goals) &
                        (nhl_df.away_goals == row.away_goals) &
                        ((nhl_df.date_time >= (row.date - timedelta(days=1))) |
                         (nhl_df.date_time <= (row.date + timedelta(days=1))))]
            print(e)
            print(row.date, row.home, row.away, row.home_goals, row.away_goals)

    result.insert(0, 'game_id', game_ids)
    
    return result

Now we can build the flattened data frames. I've also kept the unflattened data frames as they can be useful for data inspection.

### Find Marginal Stats for Each Game

So far we have:
* loaded the dataframes
* calculated the running average for each metric for each team
* converted each game into a flat structure

Now we can calculate the marginal stats for the difference between these stats.

In [7]:
def natstat_marginal(df):
    header = df.loc[:, 'game_id':'TOI'].reset_index(drop=True)
    
    h = np.core.defchararray.find(df.columns.values.astype(str), 'avg_home') >= 0
    a = np.core.defchararray.find(df.columns.values.astype(str), 'avg_away') >= 0

    homestats = df.loc[:, h]
    awaystats = df.loc[:, a]

    homestats.columns = np.char.replace(homestats.columns.values.astype(str), '_home', '')
    awaystats.columns = np.char.replace(homestats.columns.values.astype(str), '_away', '')

    marginal_stats = homestats - awaystats
    
    result = pd.concat([header.reset_index(drop=True),
                        marginal_stats.reset_index(drop=True)], axis = 1) 
    
    return result


def natstat_percents(df):
    header = df.loc[:, 'game_id':'TOI'].reset_index(drop=True)
    p = np.core.defchararray.find(df.columns.values.astype(str), '%') >= 0
    percent = df.loc[:, p]
    return pd.concat([header, percent], axis = 1)

Now, we can build a data frame with marginal stats that we will use to build the model.

In [8]:
def natstat_build_df(df_in, nhl_in, cols = None):
    df = natstat_clean(df_in)
    df_long = natstat_add_ave_stats(df)
    df_flat = natstat_flatten(df_long.sort_values(by="Game")).reset_index(drop = True)
    df_flat = add_game_ids(df_flat, nhl_in).sort_values(by = 'game_id')
    
    df_marg = natstat_marginal(df_flat).sort_values(by = 'game_id')
    
    return df_long, df_flat, df_marg

In [10]:
natstat_path = "../data/original/natstat/"
f2014 = "Games - Natural Stat TrickTeam Season Totals - 2014.csv"
f2015 = "Games - Natural Stat TrickTeam Season Totals - 2015.csv"
f2016 = "Games - Natural Stat TrickTeam Season Totals - 2016.csv"
f2017 = "Games - Natural Stat TrickTeam Season Totals - 2017.csv"
f2018 = "Games - Natural Stat TrickTeam Season Totals - 2018.csv"

natstat2014 = pd.read_csv(natstat_path + f2014)
natstat2015 = pd.read_csv(natstat_path + f2015)
natstat2016 = pd.read_csv(natstat_path + f2016)
natstat2017 = pd.read_csv(natstat_path + f2017)
natstat2018 = pd.read_csv(natstat_path + f2018)

natstat2014, natstat2014_flat, natstat2014_marg = natstat_build_df(natstat2014, g2014)
natstat2015, natstat2015_flat, natstat2015_marg = natstat_build_df(natstat2015, g2015)
natstat2016, natstat2016_flat, natstat2016_marg = natstat_build_df(natstat2016, g2016)
natstat2017, natstat2017_flat, natstat2017_marg = natstat_build_df(natstat2017, g2017)
natstat2018, natstat2018_flat, natstat2018_marg = natstat_build_df(natstat2018, g2018)

Write resulting dataframes to csv files

In [11]:
path_result = "../data/wrangled/"
natstat2016_flat.to_csv(path_result + "natstat2016_flat3.csv")
natstat2016_marg.to_csv(path_result + "natstat2016_marg3.csv")
natstat2017_flat.to_csv(path_result + "natstat2017_flat3.csv")
natstat2017_marg.to_csv(path_result + "natstat2017_marg3.csv")
natstat2018_flat.to_csv(path_result + "natstat2018_flat3.csv")
natstat2018_marg.to_csv(path_result + "natstat2018_marg3.csv")
natstat2014_marg.to_csv(path_result + "natstat2014_marg3.csv")
natstat2014_flat.to_csv(path_result + "natstat2014_flat3.csv")
natstat2015_marg.to_csv(path_result + "natstat2015_marg3.csv")
natstat2015_flat.to_csv(path_result + "natstat2015_flat3.csv")

While not strictly necessary I thought it would be interesting to look at the correlations

In [12]:
corr = natstat_percents(natstat2016_marg).corr()

# Generate a mask for the upper triangle
df = corr.where(np.tril(np.ones(corr.shape)).astype(np.bool)).fillna(0)
df.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

,game_id,home_goals,away_goals,result_bool,TOI,CF%_avg,FF%_avg,SF%_avg,GF%_avg,xGF%_avg,SCF%_avg,HDCF%_avg,HDSF%_avg,HDGF%_avg,HDSH%_avg,HDSV%_avg,MDCF%_avg,MDSF%_avg,MDGF%_avg,MDSH%_avg,MDSV%_avg,LDCF%_avg,LDSF%_avg,LDGF%_avg,LDSH%_avg,LDSV%_avg,SH%_avg,SV%_avg
game_id,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
home_goals,-0.02,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
away_goals,0.03,-0.06,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
result_bool,-0.05,0.63,-0.62,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
TOI,-0.03,-0.06,0.00,-0.01,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CF%_avg,0.04,0.10,-0.09,0.10,0.04,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
FF%_avg,0.03,0.10,-0.08,0.11,0.04,0.94,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
SF%_avg,0.03,0.09,-0.10,0.11,0.03,0.88,0.95,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GF%_avg,-0.01,0.08,-0.16,0.15,-0.04,0.13,0.21,0.27,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
xGF%_avg,0.04,0.08,-0.13,0.11,0.01,0.69,0.80,0.79,0.47,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Find Marginal Stats for Kaggle Game Data

Kaggle game statistics files contain the more traditional hockey stats like:
* power play goals
* face off wins
* hits
* etc.

We want to look at the contribution that these stats have on the final model as well.

In [ ]:
def kaggle_add_ave_stats(df_in, cols = None):
    ''' we want to create running averages for all of the numberic values of interest'''
    df = df_in.copy()
    
    # get cols to create averages of
    if cols is None:
        #cols = df.loc[:, 'CF':].columns
        cols = df.iloc[:, (df.columns.get_loc('TOI') + 1):].columns

    for c in cols:
        # create a column header name
        new_col = c + '_avg'
        
        # create the new column and fill with 0.0
        df[new_col] = 0.0
        
        # filter for each team and create the running average for each team individually
        for team in df.team_id.sort_values().unique():
            f = df['team_id'] == team
            df.loc[f, new_col] = df.loc[f, c].expanding(min_periods=2).mean().shift(1)
            
    return df

def kaggle_marginal(df):
    header = df.loc[:, 'game_id'].reset_index(drop=True)
    
    h = np.core.defchararray.find(df.columns.values.astype(str), 'avg_home') >= 0
    a = np.core.defchararray.find(df.columns.values.astype(str), 'avg_away') >= 0

    homestats = df.loc[:, h]
    awaystats = df.loc[:, a]

    homestats.columns = np.char.replace(homestats.columns.values.astype(str), '_home', '')
    awaystats.columns = np.char.replace(homestats.columns.values.astype(str), '_away', '')

    marginal_stats = homestats - awaystats
    
    z = pd.concat([header, marginal_stats], axis = 1)
    
    return z


In [ ]:
game_teams_stats = team_stats.copy()

def get_year(df, year):
    f = ((df.game_id >= (year * 1000000 + 20001)) &
         (df.game_id <  (year * 1000000 + 30001)))
    return df[f].sort_values(by='game_id')
    

cols_team = ['goals', 'shots',
             'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals',
             'faceOffWinPercentage', 'giveaways', 'takeaways']

teams_game_stats = dict()
dfTeam = dict()
dfMarg = dict()
for i in tqdm(range(2014,2019)):
    teams_game_stats[i] = get_year(game_teams_stats, i)
    dfTeam[i] = kaggle_add_ave_stats(teams_game_stats[i], cols_team)
    away_team = dfTeam[i][dfTeam[i]['HoA'] == 'away']
    home_team = dfTeam[i][dfTeam[i]['HoA'] == 'home']
    dfResult = pd.merge(away_team, home_team, on='game_id', suffixes = ['_away', '_home'])
    dfMarg[i] = kaggle_marginal(dfResult.reset_index())

We have the Natural Stattrick Marginal stats dataframes from earlier and now we have the traditional stats from Kaggle and we want to merge these into a single dataframe.

We will:
1. Concat the dataframes
2. Merge the dataframe

As we have added the game id from the Kaggle data to the Natural Stattrick data this is a simple process.

In [ ]:
natstat = pd.concat([natstat2014_marg,
                     natstat2015_marg,
                     natstat2016_marg,
                     natstat2017_marg])
dfMarg_combined = pd.concat([dfMarg[yr] for yr in [2017, 2016, 2015, 2014]])

dfCombine = pd.merge(natastat, 
                     dfMarg_combined, 
                     left_on = 'game_id',
                     right_on = 'game_id',
                     suffixes = ['_natastat', '_dfResult_tmp'])
dfCombine_2018 = pd.merge(natstat2018_marg, 
                          dfMarg[2018], 
                          left_on = 'game_id',
                          right_on = 'game_id',
                          suffixes = ['_natastat', '_dfResult_tmp'])

Finally we can write the result to a CSV for processing in *R* for the final report.

In [ ]:
dfCombine.to_csv(path_result + 'Tradition_stat_mrd10.csv')
dfCombine_2018.to_csv(path_result + 'Tradition_stat_2018_mrd10.csv')